In [ ]:
!pip install youtube_dl
!pip install SpeechRecognition

In [ ]:
import youtube_dl  # Downloading videos
import SpeechRecognition as sr  # Speech recognition
from transformers import pipeline  # Text summarization

# Download YouTube video audio only
def download_audio(url):
  ydl_opts = {
      'format': 'bestaudio/best',
      'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'wav',
          'preferredquality': '192',
      }],
  }
  with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    info_dict = ydl.extract_info(url, download=False)
    file_name = ydl.prepare_filename(info_dict)
    ydl.download([url])
  return file_name

# Transcribe audio to text
def transcribe_audio(audio_file):
  recognizer = sr.Recognizer()
  with sr.AudioFile(audio_file) as source:
    audio_data = recognizer.record(source)
  text = recognizer.recognize_google(audio_data)
  return text.lower()

# Summarize text
def summarize_text(text):
  summarizer = pipeline("summarization")
  summary = summarizer(text, max_length=100, min_length=30)  # Adjust lengths as needed
  return summary[0]['summary_text']

# Main function
def summarize_video(url):
  audio_file = download_audio(url)
  text = transcribe_audio(audio_file)
  summary = summarize_text(text)
  print(f"Summary of video: {summary}")

# Example usage
video_url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"  # Replace with your video URL
summarize_video(video_url)

ModuleNotFoundError: No module named 'SpeechRecognition'

In [ ]:
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.5 MB/s eta 0:00:00


In [ ]:
!pip install speech_recognition

ERROR: Could not find a version that satisfies the requirement speech_recognition (from versions: none)
ERROR: No matching distribution found for speech_recognition


In [ ]:
from pytube import YouTube
import speech_recognition as sr

def download_audio(url):
  """Downloads audio from a YouTube video URL."""
  try:
    yt = YouTube(url)
    audio_stream = yt.streams.filter(only_audio=True).first()
    audio_stream.download()
    return audio_stream.default_filename
  except Exception as e:
    print(f"Error downloading audio: {e}")
    return None

def speech_to_text(audio_file):
  """Converts audio file to text using an external service."""
  recognizer = sr.Recognizer()
  with sr.AudioFile(audio_file) as source:
    audio_data = recognizer.record(source)
  text = recognizer.recognize_google(audio_data)  # Replace with your service's recognition function
  return text

print(speech_to_text(download_audio('https://www.youtube.com/watch?v=QTB1YiWxxKU')))

ModuleNotFoundError: No module named 'speech_recognition'

In [ ]:
!pip install --upgrade youtube_dl

In [ ]:
import youtube_dl

def download_subtitles(url):
    ydl_opts = {
        'writesubtitles': True,
        'subtitlelangs': ['en'],  # Change to the language code you want
        'skip_download': True,
        'verbose': True,# We only want to extract subtitles, not download the video
    }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(url, download=False)
        if 'subtitles' in info_dict:
            subtitles = info_dict['subtitles']
            for lang in subtitles:
                subtitle = subtitles[lang]
                print(f"Subtitles available in {lang}: {subtitle[0]['ext']} format")
                subtitle_url = subtitle[0]['url']
                subtitle_content = ydl.urlopen(subtitle_url).read().decode('utf-8')
                # You can do anything with the subtitle content here, for example, save it to a file
                with open(f'subtitles_{lang}.srt', 'w', encoding='utf-8') as file:
                    file.write(subtitle_content)
        else:
            print("No subtitles available for this video.")

if __name__ == "__main__":
    video_url = input("Enter the URL of the YouTube video: ")
    download_subtitles(video_url)


Enter the URL of the YouTube video: https://www.youtube.com/watch?v=QTB1YiWxxKU


[debug] Encodings: locale UTF-8, fs utf-8, out UTF-8, pref UTF-8
[debug] youtube-dl version 2021.12.17
[debug] Python version 3.10.12 (CPython) - Linux-6.1.58+-x86_64-with-glibc2.35
[debug] exe versions: ffmpeg 4.4.2, ffprobe 4.4.2
[debug] Proxy map: {'colab_language_server': '/usr/colab/bin/language_service'}


[youtube] QTB1YiWxxKU: Downloading webpage


ERROR: Unable to extract uploader id; please report this issue on https://yt-dl.org/bug . Make sure you are using the latest version; see  https://yt-dl.org/update  on how to update. Be sure to call youtube-dl with the --verbose flag and include its complete output.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/youtube_dl/YoutubeDL.py", line 815, in wrapper
    return func(self, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/youtube_dl/YoutubeDL.py", line 836, in __extract_info
    ie_result = ie.extract(url)
  File "/usr/local/lib/python3.10/dist-packages/youtube_dl/extractor/common.py", line 534, in extract
    ie_result = self._real_extract(url)
  File "/usr/local/lib/python3.10/dist-packages/youtube_dl/extractor/youtube.py", line 1794, in _real_extract
    'uploader_id': self._search_regex(r'/(?:channel|user)/([^/?&#]+)', owner_profile_url, 'uploader id') if owner_profile_url else None,
  File "/usr/local/lib/python3.10/dist-pa

DownloadError: ERROR: Unable to extract uploader id; please report this issue on https://yt-dl.org/bug . Make sure you are using the latest version; see  https://yt-dl.org/update  on how to update. Be sure to call youtube-dl with the --verbose flag and include its complete output.

In [ ]:
import nltk
from pytube import YouTube
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from heapq import nlargest

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

def video_summarizer(video_url, num_sentences):
    # Download the YouTube video
    yt = YouTube(video_url)
    video = yt.streams.get_highest_resolution()
    video.download('.')

    # Extract video transcript
    transcript = yt.captions.get_by_language_code('en').generate_srt_captions()
    transcript_text = ''.join([caption[1] + ' ' for caption in transcript])

    # Tokenize sentences
    sentences = sent_tokenize(transcript_text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = nltk.word_tokenize(transcript_text)
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # Calculate word frequency
    word_freq = FreqDist(filtered_words)

    # Calculate sentence score based on word frequency
    sentence_scores = {}
    for sentence in sentences:
        for word in nltk.word_tokenize(sentence.lower()):
            if word in word_freq.keys():
                if len(sentence.split(' ')) < 30:
                    if sentence not in sentence_scores.keys():
                        sentence_scores[sentence] = word_freq[word]
                    else:
                        sentence_scores[sentence] += word_freq[word]

    # Select top sentences for summary
    summary_sentences = nlargest(num_sentences, sentence_scores, key=sentence_scores.get)
    summary = ' '.join(summary_sentences)
    return summary

# Example usage
video_url = 'https://www.youtube.com/watch?v=r2pEEkaFq8U'
num_sentences = 3
summary = video_summarizer(video_url, num_sentences)
print("Summary:")
print(summary)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-8-38059ee29386>:19: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  transcript = yt.captions.get_by_language_code('en').generate_srt_captions()


AttributeError: 'NoneType' object has no attribute 'generate_srt_captions'

In [ ]:
!pip install nltk

#Motion Detection in Video

In [ ]:
import cv2
import nltk
import numpy as np
import moviepy.editor as mp
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from heapq import nlargest

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Function to detect motion in video frames
def detect_motion(video_path):
    cap = cv2.VideoCapture(video_path)
    motion_detected_frames = []

    # Initialize background subtractor
    fgbg = cv2.createBackgroundSubtractorMOG2()

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Apply background subtraction
        fgmask = fgbg.apply(frame)

        # Count non-zero pixels in the mask
        motion_pixels = np.count_nonzero(fgmask)

        # Set a threshold for motion detection
        if motion_pixels > 100:  # Adjust threshold as needed
            motion_detected_frames.append(frame)

    cap.release()
    return motion_detected_frames

# Function to summarize text
def text_summarizer(text, num_sentences):
    # Tokenize sentences
    sentences = sent_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = nltk.word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # Calculate word frequency
    word_freq = FreqDist(filtered_words)

    # Calculate sentence score based on word frequency
    sentence_scores = {}
    for sentence in sentences:
        for word in nltk.word_tokenize(sentence.lower()):
            if word in word_freq.keys():
                if len(sentence.split(' ')) < 30:
                    if sentence not in sentence_scores.keys():
                        sentence_scores[sentence] = word_freq[word]
                    else:
                        sentence_scores[sentence] += word_freq[word]

    # Select top sentences for summary
    summary_sentences = nlargest(num_sentences, sentence_scores, key=sentence_scores.get)
    summary = ' '.join(summary_sentences)
    return summary

# Main function
def main(video_path, num_sentences):
    # Detect motion in the video
    motion_frames = detect_motion(video_path)

    # Extract audio from the video
    audio_path = 'audio.wav'
    video = mp.VideoFileClip(video_path)
    video.audio.write_audiofile(audio_path)

    # Apply text summarization on the transcript
    transcript_text = ""  # Replace with the transcribed text
    summary = text_summarizer(transcript_text, num_sentences)

    return summary

# Example usage
video_path = '/content/SampleVideo_1280x720_5mb.mp4'
num_sentences = 3
summary = main(video_path, num_sentences)
print("Summary:")
print(summary)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


MoviePy - Writing audio in audio.wav


MoviePy - Done.
Summary:

